In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import date
import numpy as np
import mne as mne
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix

ELECTRODES_NUM = 35

today = date.today().strftime("%b-%d-%Y")
root_folder = f"data/{today}"

def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)

def plot_confusion_matrix(cm, class_names,flag):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    cm_norm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm_norm.max() / 2.
    
    for i, j in product(range(cm_norm.shape[0]), range(cm_norm.shape[1])):
        color = "white" if cm_norm[i, j] > 0.34 else "black"
        plt.text(j, i, '({:.0f}%)\n {}'.format(cm_norm[i, j]*100, cm[i,j]), horizontalalignment="center", color=color, linespacing=3, fontsize='large')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

def one_hot_encoder(labels, classes_num = 3):
    epochs_num = labels.shape[0]
    labels_onehot = np.zeros((epochs_num, classes_num))
    for i in range(epochs_num):
        labels_onehot[i, int(labels[i]-1)] = 1
    return labels_onehot

def print_confusion_matrix(model, data, true_labels_onehot):
    ###### Evaluation
    prediction = model.predict(data)
    y_hat = np.zeros(np.shape(true_labels_onehot))
    for i in range(np.shape(y_hat)[0]):
        y_hat[i,np.argmax(prediction[i,:])] = 1
    delta = np.sum(abs(y_hat - true_labels_onehot), axis=1)
    accuracy = len(delta[delta==0])/len(delta)*100

    # Calculating Confusion Matrix
    y_pred = np.argmax(y_hat, axis=1)
    y_true = np.argmax(true_labels_onehot, axis=1)
    cm = confusion_matrix(y_true, y_pred)
    
    class_names=["Right","Left","No Movement"]
    figure = plot_confusion_matrix(cm, class_names=class_names, flag=1)

In [2]:
raw = mne.io.read_raw(r"C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\מאטלב\igor_data\08_22_16_Roni\Training\EEG SET FILTERED\EEG_Data_01.set")
epochs = mne.epochs(raw)
epochs
# labels = epochs.events[:,2]
# labels_onehot = one_hot_encoder(labels)
# print(labels.shape)

Reading C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\מאטלב\igor_data\08_22_16_Roni\Training\EEG SET FILTERED\EEG_Data_01.fdt


C:\Users\alonb\AppData\Local\Temp\ipykernel_38836\4197552559.py:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw(r"C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\מאטלב\igor_data\08_22_16_Roni\Training\EEG SET FILTERED\EEG_Data_01.set")


TypeError: 'module' object is not callable

In [ ]:
# Transform data
electrodes_names_grid = np.array([
    ['F5','F3','F1','Fz','F2','F4','F6'],
    ['FC5','FC3','FC1','FCz','FC2','FC4','FC6'],
    ['C5','C3','C1','Cz','C2','C4','C6'],
    ['CP5','CP3','CP1','CPz','CP2','CP4','CP6'],
    ['P5','P3','P1','Pz','P2','P4','P6']
])
ROW_NUM, COL_NUM = electrodes_names_grid.shape
TIME_SAMPLES_NUM = epochs.times.shape[0]
EPOCHS_NUM = epochs.selection.shape[0]

data_reshaped = np.empty((ROW_NUM, COL_NUM, EPOCHS_NUM, TIME_SAMPLES_NUM))

for i in range(ROW_NUM):
    for j in range(COL_NUM):
        channel_name = [electrodes_names_grid[i,j]]
        data_reshaped[i,j,:,:] = np.squeeze(epochs.copy().pick_channels(channel_name).get_data())
        
data_reshaped = np.moveaxis(data_reshaped, 2, 0)
data_reshaped = np.expand_dims(data_reshaped, axis=4)

X_train_val, X_test, y_train_val, y_test = train_test_split(data_reshaped, labels_onehot, test_size=0.2, random_state=123, stratify=labels_onehot)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=123, stratify=y_train_val)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
model = keras.Sequential([
    layers.Conv3D(20, (3,3,200), padding='valid', input_shape=(5, 7, TIME_SAMPLES_NUM, 1), activation="relu"),
    layers.Conv3D(10, (2,2,20), padding='valid', activation="relu"),
    layers.MaxPooling3D((2, 2, 50), strides=(1,1,50)),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(3, activation="softmax")
])

model.compile(optimizer='adam',
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

In [ ]:
# Plot Accuracy graph (train set and val set)
metrics_df = pd.DataFrame(history.history)
ax = metrics_df[["accuracy", "val_accuracy"]].plot(title="Accuracy");
ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")
ax.set_title("Accuracy")
ax.legend(["Training accuracy", "Validation accuracy"]);

# Plot Loss graph (train set and val set)
axe = metrics_df[["loss", "val_loss"]].plot(title="Loss");
axe.set_xlabel("Epochs")
axe.set_ylabel("Loss Value")
axe.set_title("Loss")
axe.legend(["Training loss", "Validation loss"]);

print_confusion_matrix(model, X_val, y_val)